In [60]:
import scipy.io
import numpy as np

In [61]:
traindata=scipy.io.loadmat("train_data.mat")
testdata=scipy.io.loadmat("test_data.mat")

In [62]:
traindata['label']

array([[3, 3, 3, ..., 7, 7, 7]], dtype=int64)

In [63]:
# traindatafalltened=traindata
def flattening():
    l=[]
    for i in traindata['data']:
        i=i.flatten()
        l.append(i)
    traindata['data']=np.array(l)
    l=[]
    for i in testdata['data']:
        i=i.flatten()
        l.append(i)
    testdata['data']=np.array(l)

In [64]:
flattening()

In [65]:
def skewvectorCalc(data):
    skewvector=[]
    for i in data:
        mean=np.mean(i)
        standardDeviation =np.std(i)
        sdcube=pow(standardDeviation,3)
        N=len(i)
        a=0
        for j in i:
            a+=pow(j-mean,3)
        a/=N
        a/=sdcube
        skewvector.append(a)
    return skewvector

In [66]:
def brightToDarkPixelRatio(array,threshold):
    brightdarkratio=[]
    for a in array:
        n=len(a)
        lenofbright=len([i for i in a if i>threshold])
        brightdarkratio.append(lenofbright/(n-lenofbright))
        
    return brightdarkratio

        

In [67]:
def skewBtoD_threshold_and_Nomralized(T):
    #SkewVecotor and Bright to Dark Ration Train and Test
    skewvectortrain=skewvectorCalc(traindata['data'])
    btodratiotrain=brightToDarkPixelRatio(traindata['data'],T)
    skewvectortest=skewvectorCalc(testdata['data'])
    btodratiotest=brightToDarkPixelRatio(testdata['data'],T)
    #Caluculation of Mean Variance in Train 
    skewnormalmean=np.mean(skewvectortrain)
    skewnormalstd=np.std(skewvectortrain)
    btodtnormalmean=np.mean(btodratiotrain)
    btodnormalstd=np.std(btodratiotrain)
    #Normalized Both Train and Test
    skewvectortrain=(skewvectortrain-skewnormalmean)/skewnormalstd
    btodratiotrain=(btodratiotrain-btodtnormalmean)/btodnormalstd
    skewvectortest=(skewvectortest-skewnormalmean)/skewnormalstd
    btodratiotest=(btodratiotest-btodtnormalmean)/btodnormalstd
    # Making a Matrix from Skewvector and Bright to dark ratio in train and test
    newtraindata=np.stack([skewvectortrain,btodratiotrain],axis=1)
    newtestdata=np.stack([skewvectortest,btodratiotest],axis=1)
    return newtraindata,newtestdata

In [68]:
def fetching_Disturbution_of_3_and_7(newtraindata):
    #For Number 3
    number_3_train_data=[]
    number_7_train_data=[]
    for i in range(0,len(newtraindata)):
        if traindata['label'][0][i]==3:
            number_3_train_data.append(newtraindata[i])
        if traindata['label'][0][i]==7:
            number_7_train_data.append(newtraindata[i])
    number_3_train_data=np.array(number_3_train_data)
    number_7_train_data=np.array(number_7_train_data)
    return number_3_train_data,number_7_train_data

In [69]:
def MLE_Estimation_Parameters_Mean_and_Variance(number_3_train_data,number_7_train_data):
    number_3_mean=np.mean(number_3_train_data,axis=0)
    number_3_covariance=np.cov(number_3_train_data[:,0],number_3_train_data[:,1])
    number_7_mean=np.mean(number_7_train_data,axis=0)
    number_7_covariance=np.cov(number_7_train_data[:,0],number_7_train_data[:,1])
    return number_3_mean,number_3_covariance,number_7_mean,number_7_covariance

In [70]:
def valueNormalDistrub(vector,mean,covariance,dimensions):
    vector=vector-mean
    a=1/((2*np.pi)**(dimensions/2)*(np.linalg.det(covariance)**(0.5)))
    b=(-1/2)*np.matmul(np.transpose(vector),np.matmul(np.linalg.inv(covariance),vector))
    b=np.exp(b)
    return a*b


In [71]:
def ComputeBayesianError(prior1,prior2,mean1,mean2,covariance1,covariance2,data):
    Errorval=[]
    for i in range(0,len(data)):
        p1=valueNormalDistrub(data[i],mean1,covariance1,2)
        p2=valueNormalDistrub(data[i],mean2,covariance2,2)
        p1=prior1*p1
        p2=prior2*p2
        evidence=p1+p2
        pval1=p1/evidence
        pval2=p2/evidence

        if(pval1>pval2):
            Errorval.append(pval2)
        else:
            Errorval.append(pval1)
    return np.array(Errorval)

In [74]:
Thresholds=[150,200]
for t in Thresholds:
    newtraindata,newtestdata=skewBtoD_threshold_and_Nomralized(t)
    number_3_train_data,number_7_train_data=fetching_Disturbution_of_3_and_7(newtraindata)
    number_3_mean,number_3_covariance,number_7_mean,number_7_covariance=MLE_Estimation_Parameters_Mean_and_Variance(number_3_train_data,number_7_train_data)    
    print(" Threshold : "+str(t)+" number_3_mean : "+str(number_3_mean)+" number_3_covariance : "+str(number_3_covariance)+" number_7_mean :"+str(number_7_mean)+" number_7_covariance: "+str(number_7_covariance))
    priors=[[0.5,0.5],[0.3,0.7]]
    for p in priors:
        p1=p[0]
        p2=p[1]
        trainerror=ComputeBayesianError(p1,p2,number_3_mean,number_7_mean,number_3_covariance,number_7_covariance,newtraindata)

        testerror=ComputeBayesianError(p1,p2,number_3_mean,number_7_mean,number_3_covariance,number_7_covariance,newtestdata)
        print(" Threshold : "+str(t)+" prior1 : "+str(p1)+" prior2: "+str(p2)+" Train BayesianError "+str(np.mean(trainerror)))
        print(" Threshold : "+str(t)+" prior1 : "+str(p1)+" prior2: "+str(p2)+" Test BayesianError "+str(np.mean(testerror)))

 Threshold : 150 number_3_mean : [-0.37951983  0.35854635] number_3_covariance : [[ 0.8437565  -0.91737529]
 [-0.91737529  1.10355626]] number_7_mean :[ 0.37158471 -0.35104975] number_7_covariance: [[ 0.87417217 -0.72010492]
 [-0.72010492  0.64980588]]
 Threshold : 150 prior1 : 0.5 prior2: 0.5 Train BayesianError 0.31409709381432627
 Threshold : 150 prior1 : 0.5 prior2: 0.5 Test BayesianError 0.30647387097742845
 Threshold : 150 prior1 : 0.3 prior2: 0.7 Train BayesianError 0.22237422922954495
 Threshold : 150 prior1 : 0.3 prior2: 0.7 Test BayesianError 0.22088605489469224
 Threshold : 200 number_3_mean : [-0.37951983  0.30674367] number_3_covariance : [[ 0.8437565  -0.91624532]
 [-0.91624532  1.136679  ]] number_7_mean :[ 0.37158471 -0.30033017] number_7_covariance: [[ 0.87417217 -0.72986478]
 [-0.72986478  0.68416819]]
 Threshold : 200 prior1 : 0.5 prior2: 0.5 Train BayesianError 0.30569370083304287
 Threshold : 200 prior1 : 0.5 prior2: 0.5 Test BayesianError 0.3013463267107516
 Thres